<a href="https://colab.research.google.com/github/robotics-upo/rva-course-material/blob/master/imageprocessingbasics/filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Filtering in OpenCV

In this lab session we will use the following tools:

*   **OpenCV**: http://opencv.org
*   **Numpy**, for handling multidimensional arrays (like images): https://numpy.org/
*   **Matplotlib**, library for visualization in python: https://matplotlib.org/

We will use the 4.x version of OpenCV’s API. We will intensively refer to the documentation (https://docs.opencv.org/4.11.0/d6/d00/tutorial_py_root.html)

We analyze the **linear filtering** operations in OpenCV and use them to extract features of interest like borders.



In [ ]:
#OpenCV module
import cv2

#Numpy module
import numpy as np

#We can use OpenCV in Colab, but not its functions for creating plots
#We use matplotlib for generating plots
from matplotlib import pyplot as plt
from matplotlib import cm

#We use the library scikit to read images from url
#In OpenCV, the function to read from file is cv2.imread
from skimage import io


As seen in the slides, one very common region-based operation is filtering.

Linear filters operate by convolving a **kernel** with the image.

* 2D linear filters, convolutions:
  * https://docs.opencv.org/4.11.0/d4/d13/tutorial_py_filtering.html



In [ ]:
im = io.imread('https://robotics.upo.es/~lmercab/rva/background.jpg')

plt.figure()
plt.imshow(im)



# Smoothing filters

One filter that is used typically to remove noise is an **smoothing filter**. A smoothing filter averages the region surrounding the pixel of interest.

If we want to give more importance to zones closer to a given pixel, typically a Gaussian kernel is used (the Gaussian kernel has many nice properties as well, but we will not see them now).

The (isotropic) Gaussian function in 2D (centered in the point $(k/2,k/2$)) is given by:

$g(i,j)=\frac{1}{2 \pi \sigma^2}e^{-\frac{(i-k/2)^2+(j-k/2)^2}{2\sigma^2}}$


In [ ]:
#Create a Gaussian kernel
#Size
k=11
#Sigma
sigma=1.5

gaussKernel=np.zeros(shape=(k,k))

#Apply the Gaussian 'bell' equation (see above) to determine the value of element
#of the kernel.
for i in range(k):
  for j in range(k):
    gaussKernel[i,j] = np.exp(-((i-(k-1)/2)**2+(j-(k-1)/2)**2)/(2*(sigma**2)))

#Normalize the kernel (so all element sum 1.0)
gaussKernel /= np.sum(gaussKernel)

#Show the kernel values
print(gaussKernel)

#Plot the kernel as a image
plt.figure()
plt.imshow(gaussKernel,cmap=plt.cm.gray)
plt.title('Gaussian Kernel')

#Plot the kernel in 3D
u = np.arange(0, k, 1)
v = np.arange(0, k, 1)
U, V = np.meshgrid(u, v)

fig = plt.figure()
ax = plt.axes(projection='3d')
# Plot the surface.
surf = ax.plot_surface(U, V, gaussKernel,
                       linewidth=0, antialiased=False, cmap=cm.coolwarm)
ax.set_title('Gaussian Kernel (3D view)')
# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

Applying the Gaussian kernel to an image gives the same image smoothed.

**TODO**: Play with the values of *sigma* and the kernel size *k* to see the effect on the image

In [ ]:
#Apply to the original image
#If we use a colour image, the kernel is applied to each channel
#independently
blurred_image = cv2.filter2D(im,-1,gaussKernel)

plt.figure(figsize=(14,14))
plt.subplot(1,2,1)
plt.imshow(im)
plt.title('Original')
plt.subplot(1,2,2)
plt.imshow(blurred_image)
plt.title('Blurred image')

# Shaperning Filters

The convolution operation is a linear operation. This means that the sum of several filtered images is the same as the image convolved with the sum of the kernels. This can be used to combine several kernels to obtain the desired effect.

For instance, a sharpening effect can be obtained by substracting a delta kernel (that leaves the image intact) and an averaging kernel:

In [ ]:
#Sharpening filter

#Average filter: returns the average on a 3x3 neighbourhood
average_kernel = np.ones((3,3),np.float32)/9.0

#Delta kernel: returns the value of the pixel
delta_kernel = np.array([[0,0,0],
				[0,1,0],
				[0,0,0]],np.float32)


print(average_kernel)
print(delta_kernel)

#The sharpening kernel is a combination of the other two
#We subtract the average to twice the value at a pixel
sharpening_kernel = 2*delta_kernel - average_kernel

print(sharpening_kernel)

#If we use a colour image, the kernel is applied to each channel
#independently
sharpened_image = cv2.filter2D(im,-1,sharpening_kernel)

plt.figure(figsize=(14,14))
plt.imshow(im)
plt.title('Original')

plt.figure(figsize=(14,14))
plt.imshow(sharpened_image)
plt.title('Sharpened image')


# Edge Detection Filters

One important feature in images are their borders. To extract borders, we look for regions in which the *directional derivatives* are high.

These derivatives can be extracted using filters, like the **Sobel filters**.


In [ ]:
#Sobel kernel for edge detection
filter_kernel = np.array([[-1,-2,-1],
				[0,0,0],
				[1,2,1]])

print(filter_kernel)

#Apply filter over the blue channel. -1 indicates that the pixel type in the output is the same
#as the input (in this case, uint8)
h_edges = cv2.filter2D(im[:,:,0],-1,filter_kernel)

#Type of pixels
print(h_edges.dtype)

plt.figure()
plt.imshow(h_edges,cmap=plt.cm.gray)
plt.title('Filtered')

Two points the we need to take into account:

First, now we have to be careful with the types of the images. Filtering operations will lead to outputs that typically are **real numbers**, while the pixels in the original image in this case are coded as `uint8`. If we maintain the same type, we will be loosing information. By including the parameter `cv2.CV_64F` in the call to `filter2D`, the pixels of the output will be stored as `float`.



In [ ]:
#The problem with the former case is that, as the output is uint8, negative numbers
#are truncated to 0. That is edges from bright to dark do not appear
#Alternatively, we can store the output as floats
h_edges_2 = cv2.filter2D(im[:,:,0],cv2.CV_64F,filter_kernel)

#Type of pixels
print(h_edges_2.dtype)

#Get now the absolute value
abs_h_edges64f = np.absolute(h_edges_2)

#Check the difference
plt.figure()
plt.imshow(abs_h_edges64f,cmap=plt.cm.gray)
plt.title('Filtered with Sobel Kernel')

Another **important point**: The Sobel kernel can give high values in noisy regions. Thus, the derivative is typically applied to smoothed images, that eliminate part of the noise, leading to cleaner edge responses.

**TODO**: Check the difference, and play with the Gaussian kernel above to change different values of $\sigma$. We will relate this to the concept of *scale* in future lessons.


In [ ]:
h_edges_blurred = cv2.filter2D(blurred_image[:,:,0],cv2.CV_64F,filter_kernel)

#Get now the absolute value
abs_h_edges64f_blurred = np.absolute(h_edges_blurred)

#Check the difference
plt.figure(figsize=(14,14))
plt.subplot(1,2,1)
plt.imshow(abs_h_edges64f,cmap=plt.cm.gray)
plt.title('Filtered with Sobel Kernel')
plt.subplot(1,2,2)
plt.imshow(abs_h_edges64f_blurred,cmap=plt.cm.gray)
plt.title('Filtered with Gaussian and then Sobel Kernel')

The kernels are also `numpy` arrays. By **transposing** the former kernel, we obtain a filter that "responds" to vertical edges:

In [ ]:
#Same as above, but filter with the transpose of the kernel
v_edges_2_blurred = cv2.filter2D(blurred_image[:,:,0],cv2.CV_64F,np.transpose(filter_kernel))
abs_v_edges64f_blurred = np.absolute(v_edges_2_blurred)

plt.figure()
plt.imshow(abs_v_edges64f_blurred,cmap=plt.cm.gray)
plt.title('Vertical edges kernel')


## Homework #1

Play with the thresholds and add binary operations to obtain a clean binary edge image

In [ ]:
#We can add the outputs to obtain an image
#with the response of both filters (in this case, the absolute value)
abs_edges_64f = abs_v_edges64f_blurred + abs_h_edges64f_blurred

plt.figure()
plt.imshow(abs_edges_64f,cmap=plt.cm.gray)
plt.title('Edge response')

#Check the values of abs_edges_f64. They not need to be between 0 and 255
print('Maximum', np.max(abs_edges_64f))
print('Minimum', np.min(abs_edges_64f))


#We can apply the threshold operation to extract the stongest responses
ret,binary_edges = cv2.threshold(abs_edges_64f,100,255,cv2.THRESH_BINARY)

#kernel = np.ones((3,3))
#binary_edges = cv2.erode(binary_edges,kernel)

plt.figure()
plt.imshow(binary_edges,cmap=plt.cm.gray)
plt.title('Binarized edges ')

print(binary_edges.dtype)
print('Maximum', np.max(binary_edges))
print('Minimum', np.min(binary_edges))


# Contours

Given a binary image with objects of interest, we can extract their contours using the OpenCV function `findContours`

* https://docs.opencv.org/4.11.0/d3/d05/tutorial_py_table_of_contents_contours.html

The function returns a Python list with the countours of all elements in the binary image. Each contour is an array with the pixel coordinates of the points in the contour. The function offers multiple options for dealing with contours. There are **two important parameters**:

* **Mode**: how the contours are organized. For instance:
 * `CV_RETR_EXTERNAL` retrieves only the extreme outer contours.
 * `CV_RETR_LIST`: retrieves all of the contours without establishing any hierarchical relationships
* **Method**: how the contour is extracted. For instance:
 * `CV_CHAIN_APPROX_NONE` stores absolutely all the contour points.
 * `CV_CHAIN_APPROX_SIMPLE` compresses horizontal, vertical, and diagonal segments and leaves only their end points.




In [ ]:
squares = []

#findContours require unit8 binary images
edges = np.uint8(binary_edges)

#Check the parameters of findContours
contours, _hierarchy = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

print(len(contours))

#Make a copy of the original image to draw over it
#This is important. Assignments (A = B) in numpy arrays does not create
#a new array, but a new reference to the same allocated data
im_draw = im.copy()

#We can draw the contours over the image
cv2.drawContours(im_draw, contours, -1, (255, 0, 0), 2 )

plt.figure()
plt.imshow(im_draw)
plt.title('Extracted contours ')

With the contours, then we can do many things. You can compute contour features like length, area, etc:

* https://docs.opencv.org/4.11.0/dd/d49/tutorial_py_contour_features.html

In the following example, we look for rectangles in the image
(adapted from opencv_source/samples/python2/squares.py)

In [ ]:
#Function that computes the (absolute value of the) cosine of the angle formed
#by 3 consecutive vertices
def angle_cos(p0, p1, p2):
    d1, d2 = (p0-p1).astype('float'), (p2-p1).astype('float')
    return abs( np.dot(d1, d2) / np.sqrt( np.dot(d1, d1)*np.dot(d2, d2) ) )

#contours is a Python list for all contours. Each contour is a
#numpy array of points
for cnt in contours:
  #Compute lenght
  cnt_len = cv2.arcLength(cnt, True)

  #Approximate the contour by a polygon, allowing as error a 2% of the length
  cnt = cv2.approxPolyDP(cnt, 0.02*cnt_len, True)

  #If the resultant polygon has 4 vertices and is convex it may be a rectangle
  #We also set a threshold for the area of the square (to avoid too small ones)
  if len(cnt) == 4 and cv2.contourArea(cnt) > 1000 and cv2.isContourConvex(cnt):
    #Reshape the array for convenience
    #print(cnt.shape)
    cnt = cnt.reshape(-1, 2)
    #print(cnt.shape)

    #Check if the angles of the polygong are close to 90º (cos(90) = 0)
    max_cos = np.max([angle_cos( cnt[i], cnt[(i+1) % 4], cnt[(i+2) % 4] ) for i in range(4)])
    if max_cos < 0.1:
      squares.append(cnt)

#Draw resultant contours
cv2.drawContours(im_draw, squares, -1, (0, 255, 0), 3 )

plt.figure()
plt.imshow(im_draw)
plt.title('Extracted squares ')

## Homework #2

Using the given image, extract the pixel coordinates of the corners of the black square encompasing the AruCO marker, as well as the area of the white rectangles inside the black square.

In [ ]:
#Target image
hm2 = io.imread('https://robotics.upo.es/~lmercab/rva/aruco.jpg')

plt.figure()
plt.imshow(hm2)

## Homework #3

Detect and provide the center pixel coordinates of all **yield signs** in the image.

In [ ]:
#Target image
hm2 = io.imread('https://robotics.upo.es/~lmercab/rva/doble-ceda.jpg')

plt.figure()
plt.imshow(hm2)